# CBOW word2vec model
CBOW model predicts the center word given the context words.

The input to the model is the word IDs for the context words. These word IDs are fed into a common embedding layer that is initialized with small random weights. Each word ID is transformed into a vector of size (embed_size) by the embedding layer.

Thus, each row of the input context is transformed into a matrix of size (2*window_size, embed_size) by this layer.

This is fed into a lambda layer, which computes an average of all the embeddings. This average is then fed to a dense layer, which creates a dense vector of size (vocab_size) for each row.

![CBOW model](https://i.imgur.com/iJ0H699.png)

In [ ]:
%matplotlib inline
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_distances
import matplotlib.pyplot as plt
import nltk
import numpy as np
import operator

np.random.seed(42)

BATCH_SIZE = 128
NUM_EPOCHS = 20

In [ ]:
lines = []
# https://gist.github.com/phillipj/4944029
fin = open("alice_in_wonderland.txt", "r", encoding = 'utf-8')
#############################################################################
# TODO: Write a for loop to read in the corpus.                             #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################
fin.close()

In [ ]:
# for line in fin:
#     line = line.strip()
#     if len(line) == 0:
#         continue
#     lines.append(line)

In [ ]:
sents = nltk.sent_tokenize(" ".join(lines))

#############################################################################
# TODO: Use tokenizer to tokenize the corpus.                               #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################


w_lefts, w_centers, w_rights = [], [], []
for sent in sents:
#############################################################################
# TODO: Create training data for cbow.                                      #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

In [ ]:
# tokenizer = Tokenizer(5000)  # use top 5000 words only
# tokens = tokenizer.fit_on_texts(sents)
# vocab_size = len(tokenizer.word_index) + 1

#     embedding = one_hot(sent, vocab_size)
#     triples = list(nltk.trigrams(embedding))
#     w_lefts.extend([x[0] for x in triples])
#     w_centers.extend([x[1] for x in triples])
#     w_rights.extend([x[2] for x in triples])

In [ ]:
ohe = OneHotEncoder(n_values=vocab_size)
#############################################################################
# TODO: Use one hot encoder to fit on the corpus.                           #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

model = Sequential()
model.add(Dense(300, input_shape=(Xtrain.shape[1],)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(Ytrain.shape[1]))
model.add(Activation("softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", 
              metrics=["accuracy"])
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_data=(Xtest, Ytest))

In [ ]:
# Xleft = ohe.fit_transform(np.array(w_lefts).reshape(-1, 1)).todense()
# Xright = ohe.fit_transform(np.array(w_rights).reshape(-1, 1)).todense()
# X = (Xleft + Xright) / 2.0
# Y = ohe.fit_transform(np.array(w_centers).reshape(-1, 1)).todense()
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3,
#                                                 random_state=42)

In [ ]:
# plot loss function
plt.subplot(211)
plt.title("accuracy")
plt.plot(history.history["acc"], color="r", label="train")
plt.plot(history.history["val_acc"], color="b", label="validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("loss")
plt.plot(history.history["loss"], color="r", label="train")
plt.plot(history.history["val_loss"], color="b", label="validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()

In [ ]:
# evaluate model
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

# using the word2vec model
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}

# retrieve the weights from the first dense layer. This will convert
# the input vector from a one-hot sum of two words to a dense 300 
# dimensional representation
W, b = model.layers[0].get_weights()

idx2emb = {}    
for word in word2idx.keys():
    wid = word2idx[word]
    vec_in = ohe.fit_transform(np.array(wid)).todense()
    vec_emb = np.dot(vec_in, W)
    idx2emb[wid] = vec_emb

for word in ["stupid", "alice", "succeeded"]:
    wid = word2idx[word]
    source_emb = idx2emb[wid]
    distances = []
    for i in range(1, vocab_size):
        if i == wid:
            continue
        target_emb = idx2emb[i]
        distances.append(((wid, i), 
                         cosine_distances(source_emb, target_emb)))
    sorted_distances = sorted(distances, key=operator.itemgetter(1))[0:10]
    predictions = [idx2word[x[0][1]] for x in sorted_distances]
    print("{:s} => {:s}".format(word, ", ".join(predictions)))